In [1]:
import keras
import os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K
from google.colab import drive
from sklearn.metrics import mean_squared_error
from keras.models import Model
from tensorflow.keras import regularizers
from keras.layers import Flatten, Dense, Input, Embedding, Dot
from keras.layers import GlobalAveragePooling1D, MaxPooling1D, Dropout, BatchNormalization
from keras.optimizers import Adam

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# load data
movie = [i.strip().split("::") for i in open("/content/drive/My Drive/MovieLens10M/movies.dat").readlines()]
movie = pd.DataFrame(movie)
movie = movie.rename(columns={0: "movieId", 1: "title", 2: "genres"})
rating = [i.strip().split("::") for i in open("/content/drive/My Drive/MovieLens10M/ratings.dat").readlines()]
rating = pd.DataFrame(rating)
rating = rating.rename(columns={0: "userId", 1: "movieId", 2: "rating", 3: "timestamp"})
movie = movie.loc[:,["movieId","genres"]]
rating = rating.loc[:,["userId","movieId","rating","timestamp"]]
data = pd.merge(rating, movie)
data['rating'] = pd.to_numeric(data['rating'])
data['timestamp'] = pd.to_numeric(data['timestamp'])

In [3]:
# preprocessing
n_movies = len(data['movieId'].unique())
n_users = len(data['userId'].unique())
genres_split = data.genres.str.split('|').tolist()
genres_unique = set()
for movie_genres in genres_split:
    for genre in movie_genres:
        genres_unique.add(genre)
genres2idx = {o:i+1 for i,o in enumerate(genres_unique)}
genres_split = [[genres2idx[x] for x in movie_genres] for movie_genres in genres_split]
padded_genres = tf.keras.preprocessing.sequence.pad_sequences(
    genres_split, padding="post"
)

users = data.userId.unique()
movies = data.movieId.unique()

userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

data['userId'] = data['userId'].apply(lambda x: userid2idx[x])
data['movieId'] = data['movieId'].apply(lambda x: movieid2idx[x])

data = data.drop(columns=['genres'])
for i in range(len(padded_genres[0])):
    data['genre'+str(i)] = padded_genres[:,i]

min_timestamp = pd.DataFrame(data.loc[:,["movieId","timestamp"]].groupby(['movieId'], as_index=False).min())
min_timestamp = min_timestamp.sort_values(by=["movieId"], axis=0)
min_timestamp = min_timestamp.rename(columns={"timestamp": "min_timestamp"})
data = pd.merge(data, min_timestamp)
data['timestamp'] = data['timestamp'] - data['min_timestamp']
data['timestamp'] = data['timestamp'] / (60 * 60 * 24 * 365)
data = data.drop(columns=['min_timestamp'])
data['day'] = data['timestamp'] * 365 / (7 * 2)
data['day'] = data['day'].apply(lambda x: int(x))
days = data.day.unique()
n_days = len(days)
day2idx = {o:i for i,o in enumerate(days)}
data['day'] = data['day'].apply(lambda x: day2idx[x])
print(n_movies, n_users, len(rating), n_days)

10677 69878 10000054 365


In [4]:
# split train and test data
split = np.random.rand(len(data)) < 0.9
train = data[split]
valid = data[~split]
data.head(10)

,userId,movieId,rating,timestamp,genre0,genre1,genre2,genre3,genre4,genre5,genre6,genre7,day
0,0,0,5.0,0.347135,19,8,0,0,0,0,0,0,0
1,1,0,3.0,4.638027,19,8,0,0,0,0,0,0,1
2,2,0,2.5,9.015238,19,8,0,0,0,0,0,0,2
3,3,0,3.0,3.659976,19,8,0,0,0,0,0,0,3
4,4,0,4.5,8.702935,19,8,0,0,0,0,0,0,4
5,5,0,3.0,0.519782,19,8,0,0,0,0,0,0,5
6,6,0,3.0,0.520016,19,8,0,0,0,0,0,0,5
7,7,0,3.0,0.347532,19,8,0,0,0,0,0,0,0
8,8,0,1.0,4.052296,19,8,0,0,0,0,0,0,6
9,9,0,3.0,3.736893,19,8,0,0,0,0,0,0,7


In [13]:
# generate model
embedding_l2 = 1e-6 * 2                                  # regularization
dense_l2 = 1e-3 * 2                                      # regularization
keras.backend.clear_session()

user_input = Input(shape=(1,), name='user_input', dtype='int64')
movie_input = Input(shape=(1,), name='movie_input', dtype='int64')
genres_input = Input(shape=(len(padded_genres[0]),), name='genres_input', dtype='int64') # input = (10, 1)
day_input = Input(shape=(1,), name='day_input', dtype='int64')
time_input = Input(shape=(1,), name='time_input', dtype='float32')
time_root = tf.math.sqrt(time_input + 1.0)
time_square = tf.math.square(time_input)
time_vector = tf.concat([time_input, time_root, time_square], 1)

def create_model(n_user_embedding):
    n_movie_embedding = int(n_user_embedding * 0.75)
    n_genres_embedding = int(n_user_embedding * 0.25)

    user_embedding = Embedding(n_users, n_user_embedding, embeddings_regularizer=regularizers.l2(embedding_l2))(user_input)
    user_vector = Flatten()(user_embedding)
    movie_embedding = Embedding(n_movies, n_movie_embedding, embeddings_regularizer=regularizers.l2(embedding_l2))(movie_input)
    movie_vector = Flatten()(movie_embedding)
    genres_embedding = Embedding(len(genres_unique)+1, n_genres_embedding, mask_zero=True, embeddings_regularizer=regularizers.l2(embedding_l2/(len(genres_unique)+1)))(genres_input)
    genres_average_embedding = GlobalAveragePooling1D()(genres_embedding)
    genres_vector = Flatten()(genres_average_embedding)
    day_embedding = Embedding(n_days, int(n_user_embedding/8), embeddings_regularizer=regularizers.l2(embedding_l2))(day_input)
    day_vector = Flatten()(day_embedding)

    concat_layer = tf.concat([user_vector, movie_vector, genres_vector, time_vector, day_vector], 1)
    mlp_layer = Dense(int(n_user_embedding/2), activation='relu', kernel_regularizer=regularizers.l2(dense_l2))(concat_layer)

    user_embedding2 = Embedding(n_users, n_user_embedding, embeddings_regularizer=regularizers.l2(embedding_l2))(user_input)
    user_vector2 = Flatten()(user_embedding2)
    movie_embedding2 = Embedding(n_movies, n_movie_embedding, embeddings_regularizer=regularizers.l2(embedding_l2))(movie_input)
    movie_vector2 = Flatten()(movie_embedding2)
    genres_embedding2 = Embedding(len(genres_unique)+1, n_genres_embedding, mask_zero=True, embeddings_regularizer=regularizers.l2(embedding_l2/(len(genres_unique)+1)))(genres_input)
    genres_average_embedding2 = GlobalAveragePooling1D()(genres_embedding2)
    genres_vector2 = Flatten()(genres_average_embedding2)
    matrix_factorization = Dot(axes=1)([user_vector2, tf.concat([movie_vector2, genres_vector2], 1)])

    user_embedding3 = Embedding(n_users, n_user_embedding, embeddings_regularizer=regularizers.l2(embedding_l2))(user_input)
    user_vector3 = Flatten()(user_embedding3)
    movie_embedding3 = Embedding(n_movies, n_movie_embedding, embeddings_regularizer=regularizers.l2(embedding_l2))(movie_input)
    movie_vector3 = Flatten()(movie_embedding3)
    genres_embedding3 = Embedding(len(genres_unique)+1, n_genres_embedding, mask_zero=True, embeddings_regularizer=regularizers.l2(embedding_l2/(len(genres_unique)+1)))(genres_input)
    genres_average_embedding3 = GlobalAveragePooling1D()(genres_embedding3)
    genres_vector3 = Flatten()(genres_average_embedding3)
    diff = tf.math.subtract(user_vector3, tf.concat([movie_vector3, genres_vector3], 1))
    W = tf.Variable(tf.random.normal(shape=[n_user_embedding, 1], stddev=0.1))
    diff = tf.linalg.matmul(diff, W)

    output_layer = tf.concat([matrix_factorization, mlp_layer, diff], 1)
    return output_layer

output_layer_64 = create_model(64)
output_layer_32 = create_model(32)
output_layer_16 = create_model(16)
output_layer_8 = create_model(8)
output_layer = tf.concat([output_layer_64, output_layer_32, output_layer_16, output_layer_8], 1)
output = Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(dense_l2))(output_layer)
output = output*5.5

model = Model([user_input, movie_input, time_input, genres_input, day_input], output)
model.summary()

The following Variables were used a Lambda layer's call (tf.linalg.matmul), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(64, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.linalg.matmul_1), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(32, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.linalg.matmul_2), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(16, 1) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an

In [14]:
# training
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(optimizer=Adam(), loss='mse', metrics=[rmse])
batch_size = 4096*8
epochs = 128

callback = tf.keras.callbacks.EarlyStopping(monitor='val_rmse', patience=1, restore_best_weights=True)
history = model.fit([train.userId, train.movieId, train.timestamp, train.iloc[:,4:12], train.day], train.rating,
                batch_size=batch_size, epochs=epochs, callbacks=[callback],
                validation_data = ([valid.userId, valid.movieId, valid.timestamp, valid.iloc[:,4:12], valid.day], valid.rating), verbose = 1)

Epoch 1/128
275/275 [==============================] - 198s 625ms/step - loss: 1.0736 - rmse: 0.9516 - val_loss: 0.7857 - val_rmse: 0.8603
Epoch 2/128
275/275 [==============================] - 166s 604ms/step - loss: 0.7568 - rmse: 0.8438 - val_loss: 0.7108 - val_rmse: 0.8145
Epoch 3/128
275/275 [==============================] - 167s 606ms/step - loss: 0.6696 - rmse: 0.7864 - val_loss: 0.6877 - val_rmse: 0.7953
Epoch 4/128
275/275 [==============================] - 164s 597ms/step - loss: 0.6157 - rmse: 0.7459 - val_loss: 0.6810 - val_rmse: 0.7853
Epoch 5/128
275/275 [==============================] - 162s 590ms/step - loss: 0.5673 - rmse: 0.7062 - val_loss: 0.6871 - val_rmse: 0.7832
Epoch 6/128
275/275 [==============================] - 165s 600ms/step - loss: 0.5233 - rmse: 0.6678 - val_loss: 0.7009 - val_rmse: 0.7869


In [15]:
# check final RMSE
valid_pred = model.predict([valid.userId, valid.movieId, valid.timestamp, valid.iloc[:,4:12], valid.day], batch_size = batch_size)
valid_pred = [max(min(x, 5), 0.5) for x in valid_pred]
test_rmse = mean_squared_error(valid.rating, valid_pred, squared=False) # squared=False -> RMSE
print(test_rmse)

0.7825967212279217


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
